In [1]:
import numpy as np 
import math

In [2]:
def angle_between(p1, p2):
    ang1 = np.arctan2(*p1[::-1])
    ang2 = np.arctan2(*p2[::-1])
    return np.rad2deg((ang1 - ang2) % (2 * np.pi))

In [3]:
def distance(p1, p2):
    d_x = p1[0] - p2[0]
    d_y = p1[1] - p2[1]
    return math.hypot(d_x, d_y)

In [8]:
import glob
import cv2

In [123]:
def get_anchor_points(img):
    templates = glob.glob('./data/templates/**/*.png', recursive=True)
    for template in templates:
        anchor = cv2.imread(template, cv2.IMREAD_GRAYSCALE)
        res = cv2.matchTemplate(img, anchor, cv2.TM_CCOEFF_NORMED)
        locs = np.where(res >= 0.8)
        points = zip(*locs[::-1])
        allpoints = []
        min_x, min_y = img.shape[::-1]
        max_y = 0
        for point in points:
            valid = True
            for v_point in allpoints:
                if distance(v_point, point) < 100:
                    valid = False
            if valid and point[0] < 600:
                allpoints.append(point)
                if point[0] < min_x:
                    min_x = point[0]
                if point[1] < min_y:
                    min_y = point[1]
                if point[1] > max_y:
                    max_y = point[1]
        
        if (len(allpoints) == 2):
            return allpoints, min_x, min_y, max_y
    
    return [], 0, 0, 0

In [124]:
from os import path

In [127]:
def enframe_image(file):
    try:
        img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)        
        allpoints, min_x, min_y, max_y = get_anchor_points(img)

        if len(allpoints) != 2:
            print('File invalid:', len(allpoints), file)
            return
        
        points = []
        for point in allpoints:
            p = (point[0] - min_x, max_y - min_y)
            points.append(p)

        rows, cols = img.shape
        cv2.rectangle(img, (0,0), (cols, rows), (255,255,255), 4)
        angle = angle_between(points[1], points[0])        
        rotate_m = cv2.getRotationMatrix2D((rows, cols), angle, 1)
        img = cv2.warpAffine(img, rotate_m, (cols, rows))
        move_m = np.float32([[1,0,-5], [0,1,0]])
        img = cv2.warpAffine(img, move_m, (cols, rows))
        file = path.split(file)[1]
        cv2.imwrite('./data/enframed/{0}'.format(file), img)
    except Exception as e:
        print(e)        
        pass

In [129]:
files = glob.glob('./data/raw-splited/**/*.png', recursive=True)  
for file in files:
    ck_file = path.split(file)[1]
    if path.isfile('./data/enframed/{0}'.format(ck_file)):
        continue
    enframe_image(file)

File invalid: 0 ./data/raw-splited/template-card-1071_0.png
File invalid: 0 ./data/raw-splited/template-card-803_18.png
File invalid: 0 ./data/raw-splited/template-card-803_10.png
File invalid: 0 ./data/raw-splited/template-card-348_6.png
File invalid: 0 ./data/raw-splited/template-card-1062_9.png
File invalid: 0 ./data/raw-splited/template-card-1062_5.png
File invalid: 0 ./data/raw-splited/template-card-803_2.png
File invalid: 0 ./data/raw-splited/template-card-1062_15.png
File invalid: 0 ./data/raw-splited/template-card-348_31.png
File invalid: 0 ./data/raw-splited/template-card-803_11.png
File invalid: 0 ./data/raw-splited/template-card-803_16.png
File invalid: 0 ./data/raw-splited/template-card-1062_10.png
File invalid: 0 ./data/raw-splited/template-card-623_0.png
File invalid: 0 ./data/raw-splited/template-card-1062_12.png
File invalid: 0 ./data/raw-splited/template-card-1062_21.png
File invalid: 0 ./data/raw-splited/template-card-803_14.png
File invalid: 0 ./data/raw-splited/temp